In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import os
from tqdm.notebook import tqdm as tqdm
from sklearn.manifold import TSNE
import shutil
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torchvision.models import mobilenet_v2
import time
import timeit

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:05<00:00, 29097538.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Load pre-trained MobileNetV2 model
model = mobilenet_v2(pretrained=False)

# Modify last layer to match CIFAR-10 output classes
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 10)

# Move model to device
model = model.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
def train(model, device, train_loader, optimizer, epoch, reg_strength=0.001):
    model.train()
    train_loss = 0
    train_correct_top1 = 0
    train_correct_top5 = 0
    num_batches = len(train_loader)
   
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)

        # L2 regularization
        reg_loss = sum(param.norm(2) ** 2 for param in model.parameters())
        loss += reg_strength * reg_loss

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Calculate top-1 and top-5 accuracy
        _, predicted = torch.topk(output, k=5, dim=1)
        train_correct_top1 += torch.sum(predicted[:, 0] == target)
        train_correct_top5 += torch.sum(predicted == target.unsqueeze(1))

    train_loss /= len(train_loader.dataset)
    train_acc_top1 = train_correct_top1.double() / len(train_loader.dataset)
    train_acc_top5 = train_correct_top5.double() / len(train_loader.dataset)
    return train_loss, train_acc_top1, train_acc_top5
    
def test(model, device, test_loader, reg_strength=0.001):
    model.eval()
    test_loss = 0
    test_correct = 0
    test_total = 0
    test_correct_top5 = 0
    num_batches = len(test_loader)
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction='sum')

            # L2 regularization
            reg_loss = sum(param.norm(2) ** 2 for param in model.parameters())
            loss += reg_strength * reg_loss

            test_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            test_total += target.size(0)
            test_correct += (predicted == target).sum().item()

            # Top-5 accuracy
            _, top5_predicted = torch.topk(output.data, 5, dim=1)
            for i in range(len(target)):
                if target[i] in top5_predicted[i]:
                    test_correct_top5 += 1

  
    test_loss /= len(test_loader.dataset)
    test_acc = test_correct / test_total
    test_acc_top5 = test_correct_top5 / test_total
    return test_loss, test_acc, test_acc_top5


In [ ]:
optimizer = optim.Adam(model.parameters(),lr=3e-4)
criterion = nn.CrossEntropyLoss()

In [ ]:
# He weight initialization
for name, param in model.named_parameters():
    if 'weight' in name:
        if len(param.shape) < 2:
          continue
        nn.init.kaiming_normal_(param)

In [ ]:
test_losses = []
test_accuracies_top1 = []
test_accuracies_top5 = []
train_losses = []
train_accuracies_top1 = []
train_accuracied_top5 = []
num_epochs = 15
# Train and test the model
start_time = timeit.default_timer()
for epoch in range(num_epochs):
    train_loss, train_acc_top1, train_acc_top5 = train(model, device, trainloader, optimizer, epoch)
    test_loss, test_acc_top1, test_acc_top5 = test(model, device, testloader)
    train_losses.append(train_loss)
    train_accuracies_top1.append(train_acc_top1)
    train_accuracied_top5.append(train_acc_top5)
    test_losses.append(test_loss)
    test_accuracies_top1.append(test_acc_top1)
    test_accuracies_top5.append(test_acc_top5)
    print()
    print("Epoch: ", epoch + 1, "Train Loss: ", train_loss, "Train Acc top 1: ", train_acc_top1.item(), "Test Loss: ", test_loss, "Test Acc top 1: ", test_acc_top1)
    print("Train Acc top 5: ", train_acc_top5.item(), "Test Acc top 5: ", test_acc_top5)
    print()
    if (epoch==9):
        elapsed_time_1 = timeit.default_timer() - start_time
    if (epoch==14):
        elapsed_time_2 = timeit.default_timer() - start_time

print('time for 10 epochs : ', elapsed_time_1)
print('time for 15 epochs : ', elapsed_time_2)


train_losses = torch.tensor(train_losses)
train_accuracies_top1 = torch.tensor(train_accuracies_top1)
train_accuracied_top5 = torch.tensor(train_accuracied_top5)
test_losses = torch.tensor(test_losses)
test_accuracies_top1 = torch.tensor(test_accuracies_top1)
test_accuracies_top5 = torch.tensor(test_accuracies_top5)

train_losses = train_losses.cpu().numpy()
train_accuracies_top1 = train_accuracies_top1.cpu().numpy()
train_accuracied_top5 = train_accuracied_top5.cpu().numpy()
test_losses = test_losses.cpu().numpy()
test_accuracies_top1 = test_accuracies_top1.cpu().numpy()
test_accuracies_top5 = test_accuracies_top5.cpu().numpy()



Epoch:  1 Train Loss:  0.38963901100158693 Train Acc top 1:  0.15696000000000002 Test Loss:  2.494682975769043 Test Acc top 1:  0.2005
Train Acc top 5:  0.63024 Test Acc top 5:  0.7072


Epoch:  2 Train Loss:  0.33342570610046385 Train Acc top 1:  0.2053 Test Loss:  2.320622109222412 Test Acc top 1:  0.2359
Train Acc top 5:  0.73336 Test Acc top 5:  0.764


Epoch:  3 Train Loss:  0.2829231996917725 Train Acc top 1:  0.24450000000000002 Test Loss:  2.1929703285217284 Test Acc top 1:  0.2651
Train Acc top 5:  0.7853000000000001 Test Acc top 5:  0.8099


Epoch:  4 Train Loss:  0.23814379581451417 Train Acc top 1:  0.28606000000000004 Test Loss:  2.057152237701416 Test Acc top 1:  0.307
Train Acc top 5:  0.8215600000000001 Test Acc top 5:  0.8394


Epoch:  5 Train Loss:  0.19956066120147706 Train Acc top 1:  0.32406 Test Loss:  1.9423252853393556 Test Acc top 1:  0.3451
Train Acc top 5:  0.84904 Test Acc top 5:  0.8594


Epoch:  6 Train Loss:  0.16710930835723878 Train Acc top 1:  0.35592

In [ ]:
# fig, axs = plt.subplots(1, 3, figsize=(15, 4))

# # Plot train and test loss
# axs[0].plot(train_losses,'b-', label='Train Loss')
# axs[0].plot(test_losses,'r-', label='Test Loss')
# axs[0].set_title('Train v/s Test loss')
# axs[0].legend()

# # Plot top 1 train and test accuracy
# axs[1].plot(train_accuracies_top1,'b-', label='Train Acc top 1')
# axs[1].plot(test_accuracies_top1,'r-', label='Test Acc top 1')
# axs[1].set_title('Train v/s Test accuracy (top 1)')
# axs[1].legend()

# # Plot top 5 train and test accuracy
# axs[2].plot(train_accuracied_top5,'b-', label='Train Acc top 5')
# axs[2].plot(test_accuracies_top5,'r-', label='Test Acc top 5')
# axs[2].set_title('Train v/s Test accuracy (top 5)')
# axs[2].legend()

# plt.tight_layout()
# plt.show()


In [ ]:
print('---------------- NEW HYPERPARAMETERS ----------------')

---------------- NEW HYPERPARAMETERS ----------------


In [ ]:
optimizer = optim.Adam(model.parameters(),lr=2e-3, weight_decay = 5e-4)
criterion = nn.CrossEntropyLoss()

In [ ]:
# He weight initialization
for name, param in model.named_parameters():
    if 'weight' in name:
        if len(param.shape) < 2:
          continue
        nn.init.kaiming_normal_(param)

In [ ]:
test_losses = []
test_accuracies_top1 = []
test_accuracies_top5 = []
train_losses = []
train_accuracies_top1 = []
train_accuracied_top5 = []
num_epochs = 10
# Train and test the model
# Train and test the model
start_time = timeit.default_timer()
for epoch in range(num_epochs):
    train_loss, train_acc_top1, train_acc_top5 = train(model, device, trainloader, optimizer, epoch)
    test_loss, test_acc_top1, test_acc_top5 = test(model, device, testloader)
    train_losses.append(train_loss)
    train_accuracies_top1.append(train_acc_top1)
    train_accuracied_top5.append(train_acc_top5)
    test_losses.append(test_loss)
    test_accuracies_top1.append(test_acc_top1)
    test_accuracies_top5.append(test_acc_top5)
    print()
    print("Epoch: ", epoch + 1, "Train Loss: ", train_loss, "Train Acc top 1: ", train_acc_top1.item(), "Test Loss: ", test_loss, "Test Acc top 1: ", test_acc_top1)
    print("Train Acc top 5: ", train_acc_top5.item(), "Test Acc top 5: ", test_acc_top5)
    print()

train_losses = torch.tensor(train_losses)
train_accuracies_top1 = torch.tensor(train_accuracies_top1)
train_accuracied_top5 = torch.tensor(train_accuracied_top5)
test_losses = torch.tensor(test_losses)
test_accuracies_top1 = torch.tensor(test_accuracies_top1)
test_accuracies_top5 = torch.tensor(test_accuracies_top5)

train_losses = train_losses.cpu().numpy()
train_accuracies_top1 = train_accuracies_top1.cpu().numpy()
train_accuracied_top5 = train_accuracied_top5.cpu().numpy()
test_losses = test_losses.cpu().numpy()
test_accuracies_top1 = test_accuracies_top1.cpu().numpy()
test_accuracies_top5 = test_accuracies_top5.cpu().numpy()


Training Epoch 0:   0%|          | 0/391 [00:00<?, ?batch/s]

Testing:   0%|          | 0/79 [00:00<?, ?batch/s]


Epoch:  1 Train Loss:  0.24168628744125367 Train Acc top 1:  0.25978 Test Loss:  1.9897946804046631 Test Acc top 1:  0.306
Train Acc top 5:  0.7848600000000001 Test Acc top 5:  0.8294



Training Epoch 1:   0%|          | 0/391 [00:00<?, ?batch/s]

Testing:   0%|          | 0/79 [00:00<?, ?batch/s]

...

Testing:   0%|          | 0/79 [00:00<?, ?batch/s]


Epoch:  9 Train Loss:  0.011182588126659393 Train Acc top 1:  0.66742 Test Loss:  1.0300886478424072 Test Acc top 1:  0.6422
Train Acc top 5:  0.9684200000000001 Test Acc top 5:  0.9608



Training Epoch 9:   0%|          | 0/391 [00:00<?, ?batch/s]

Testing:   0%|          | 0/79 [00:00<?, ?batch/s]


Epoch:  10 Train Loss:  0.010429350984096527 Train Acc top 1:  0.6813800000000001 Test Loss:  0.9665188865661621 Test Acc top 1:  0.6699
Train Acc top 5:  0.9699200000000001 Test Acc top 5:  0.9694

